In [ ]:
import os
import gc
import ot
import pickle
import anndata
import scanpy as sc
import squidpy as sq
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.stats import spearmanr, pearsonr
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import plotly
import commot as ct

## **Load data**

In [ ]:
#Load in the anndata object generated by running Post-RCTD Analyses.ipynb
adata_ss = anndata.read_h5ad('Puck_RCTD_Anndata.h5ad')
adata_ss

In [ ]:
adata_ss.var_names_make_unique()
sc.pp.normalize_total(adata_ss, inplace=True)
sc.pp.log1p(adata_ss)
adata_ss

In [ ]:
sc.pp.highly_variable_genes(adata_ss, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata_ss[:, adata_ss.var.highly_variable]
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.4)
adata_ss.obs['leiden'] = adata.obs['leiden']
adata_ss

## **cell-cell communication network**

In [ ]:
df_cellchat = ct.pp.ligand_receptor_database(species='human')
df_cellchat_filtered = ct.pp.filter_lr_database(df_cellchat, adata_ss, min_cell_pct=0.0)
print(df_cellchat_filtered.shape)
df_cellchat_filtered.head(3)

In [ ]:
ct.tl.spatial_communication(adata_ss,
    database_name='cellchat', df_ligrec=df_cellchat_filtered, dis_thr=200, heteromeric=True, pathway_sum=True)

## **Communication directions**

In [ ]:
# rank the pathways by activity
def rank_signaling_activity(
    adata,
    database_name = None
):
    df_ligrec = adata.uns['commot-%s-info' % database_name]['df_ligrec']
    pathways = list(set(list(df_ligrec.iloc[:,2])))
    lr_pair_names = []
    for i in range(df_ligrec.shape[0]):
        lr_pair_names.append('%s-%s' % (df_ligrec.iloc[i,0], df_ligrec.iloc[i,1]))
    act_pathway = np.zeros([len(pathways)], float)
    act_lrpairs = np.zeros([len(lr_pair_names)], float)
    for i in range(len(act_pathway)):
        act_pathway[i] = np.sum(adata.obsm['commot-%s-sum-receiver' % database_name]['r-%s' % pathways[i]].values)
    for i in range(len(act_lrpairs)):
        act_lrpairs[i] = np.sum(adata.obsm['commot-%s-sum-receiver' % database_name]['r-%s' % lr_pair_names[i]].values)
    idx_pathway = np.argsort(-act_pathway)
    idx_lrpairs = np.argsort(-act_lrpairs)
    pathways = np.array(pathways, str)
    lr_pair_names = np.array(lr_pair_names, str)
    
    return pathways[idx_pathway], lr_pair_names[idx_lrpairs]

In [ ]:
# export the pathway activity

database_name = 'cellchat'

df_ligrec = adata_ss.uns['commot-%s-info' % database_name]['df_ligrec']
pathways = list(set(list(df_ligrec.iloc[:,2])))
    
act_pathway = np.zeros([len(pathways)], float)
    
for i in range(len(act_pathway)):
    act_pathway[i] = np.sum(adata_ss.obsm['commot-%s-sum-receiver' % database_name]['r-%s' % pathways[i]].values)
    
    
pathways = np.array(pathways, str)
    
df = pd.DataFrame({'Pathway':pathways, 'Receiver_Activity':act_pathway})
df= df.sort_values(by='Receiver_Activity', ascending=False)
df.to_csv('Puck_Pathway_activity.csv')
print(df.shape)
df.head(3)

In [ ]:
ranked_pathway, ranked_lrpair = rank_signaling_activity(adata_ss, database_name='cellchat')

In [ ]:
from numpy import savetxt
savetxt('ranked_lrpair.csv', ranked_lrpair, fmt='%s')
ranked_lrpair

In [ ]:
ct.tl.communication_direction(adata_ss, database_name='cellchat')

In [ ]:
for pathway in ['signaling pathways of interest']:
    ct.tl.communication_direction(adata_ss, database_name='cellchat', pathway_name=pathway)
    ct.pl.plot_cell_communication(adata_ss, database_name='cellchat', pathway_name=pathway, scale=0.000035, interp_k=20, ndsize=1.2, grid_density=0.6,plot_method='grid', clustering='cluster', background='cluster', summary = 'receiver',cmap=plotly.colors.qualitative.Alphabet, background_legend=True,
                             filename='./figure/ccc_dir-%s.eps' % pathway)
    plt.clf()

In [ ]:
adata_ss.write('Puck_COMMOT_Anndata.h5ad')